<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Colas M/M/2

En esta sección analizamos la simulación de un sistema **M/M/2**, es decir, un modelo
de colas con:

- Llegadas Poisson de tasa \\( \lambda \\)
- Tiempos de servicio exponenciales para cada servidor
- Dos servidores trabajando en paralelo
- Una sola fila compartida
- Disciplina FIFO (First In, First Out)

El objetivo es:

1. Simular el comportamiento del sistema.
2. Obtener métricas como tiempos promedio en fila y en el sistema.
3. Comparar los resultados con las soluciones analíticas del modelo M/M/2.

In [328]:
import random
import math

In [329]:
def simular_MM2(lmbda, mu1, mu2, T_FIN):

    # Estado inicial
    t  = 0.0
    NLL = 0
    C1  = 0
    C2  = 0

    s1 = 0
    s2 = 0
    cola = []

    # tiempos de eventos
    t_LL = random.expovariate(lmbda)
    t1 = math.inf
    t2 = math.inf

    # registros
    LL = {}
    S = {}
    # Diccionario para guardar cuándo inician servicio
    Inicio = {}

    def servicio(mu):
        return random.expovariate(mu)

    # LOOP principal
    while t < T_FIN:
        # Elegimos próximo evento
        if t_LL <= t1 and t_LL <= t2:
            evento = "llegada"
        elif t1 <= t2 and t1 < t_LL:
            evento = "fin1"
        else:
            evento = "fin2"

        # LLEGADA
        if evento == "llegada":
            t = t_LL
            NLL += 1
            cliente = NLL
            LL[cliente] = t

            t_LL = t + random.expovariate(lmbda)

            if s1 == 0:
                s1 = cliente
                # Cliente entra directo a S1
                Inicio[cliente] = t
                t1 = t + servicio(mu1)

            elif s2 == 0:
                s2 = cliente
                # Cliente entra directo a S2
                Inicio[cliente] = t
                t2 = t + servicio(mu2)

            else:
                cola.append(cliente)

        # FIN SERVICIO S1
        elif evento == "fin1":
            t = t1
            if s1 != 0:
                cliente = s1
                C1 += 1
                S[cliente] = t

                if cola:
                    s1 = cola.pop(0)
                    # Cliente sale de fila y entra a S1
                    Inicio[s1] = t
                    t1 = t + servicio(mu1)
                else:
                    s1 = 0
                    t1 = math.inf

        # FIN SERVICIO S2
        else:
            t = t2

            if s2 != 0:
                cliente = s2
                C2 += 1
                S[cliente] = t

                if cola:
                    s2 = cola.pop(0)
                    # Cliente sale de fila y entra a S2
                    Inicio[s2] = t
                    t2 = t + servicio(mu2)
                else:
                    s2 = 0
                    t2 = math.inf

    if S:
        # Tiempo total en sistema
        tiempos_sistema = [S[i] - LL[i] for i in S]
        tiempo_medio_sistema = sum(tiempos_sistema) / len(tiempos_sistema)

        # Tiempo en fila
        # Solo consideramos los clientes que terminaron su servicio (están en S)
        tiempos_fila = [Inicio[i] - LL[i] for i in S]
        tiempo_medio_fila = sum(tiempos_fila) / len(tiempos_fila)

    else:
        tiempo_medio_sistema = 0
        tiempo_medio_fila = 0

    return {
        "tiempo_final": t,
        "llegadas": NLL,
        "servidos_S1": C1,
        "servidos_S2": C2,
        "LL": LL,
        "S": S,
        "tiempo_medio_sistema": tiempo_medio_sistema,
        "tiempo_medio_fila": tiempo_medio_fila
    }

In [330]:
res = simular_MM2(lmbda=1/3, mu1=1/2, mu2=1/2, T_FIN=50)

print(f"Tiempo final simulado: {res['tiempo_final']:.4f}")
print(f"Llegadas totales: {res['llegadas']}")
print(f"Clientes atendidos por S1: {res['servidos_S1']}")
print(f"Clientes atendidos por S2: {res['servidos_S2']}")
print(f"Tiempo medio en el sistema: {res['tiempo_medio_sistema']:.4f}")

Tiempo final simulado: 52.3380
Llegadas totales: 16
Clientes atendidos por S1: 8
Clientes atendidos por S2: 7
Tiempo medio en el sistema: 2.3230


In [331]:
def soluciones_analiticas_MM2(lmbda, mu):

    # 1. Intensidad de tráfico
    c = 2
    rho = lmbda / (c * mu)

    print(f"Comparación con ejercicio analítico:")
    print(f"Lambda: {lmbda:.4f}")
    print(f"Mu (cada servidor): {mu:.4f}")
    print(f"Rho (intensidad): {rho:.4f}")

    sumatoria = 1 + (lmbda/mu)
    termino_final = ((lmbda/mu)**2) / (2 * (1 - rho))

    P0 = 1 / (sumatoria + termino_final)

    # Lq = [ P0 * (lambda/mu)^c * rho ] / [ c! * (1-rho)^2 ]
    Lq = (P0 * ((lmbda/mu)**2) * rho) / (2 * (1 - rho)**2)

    # 4. Tiempo promedio en fila (Wq) usando Little: Wq = Lq / lambda
    Wq = Lq / lmbda

    # 5. Tiempo promedio en sistema (W) = Wq + 1/mu
    W = Wq + (1 / mu)

    return {
        "W_q": Wq,
        "W": W,
        "P0": P0,
        "Lq": Lq
    }

# Usamos los mismos parámetros que en la simulación
lmbda_val = 1/3
mu_val = 1/2

teoria = soluciones_analiticas_MM2(lmbda_val, mu_val)

if teoria:
    print("="*40)
    print(f"{'MÉTRICA':<20} | {'SIMULACIÓN':<10} | {'TEÓRICO':<10}")
    print("="*40)

    # Extraemos valores de la simulación (del diccionario 'res' que calculaste antes)
    sim_Wq = res['tiempo_medio_fila']
    sim_W  = res['tiempo_medio_sistema']

    # Valores teóricos
    teo_Wq = teoria['W_q']
    teo_W  = teoria['W']

    print(f"{'Tiempo en Fila (Wq)':<20} | {sim_Wq:.4f}     | {teo_Wq:.4f}")
    print(f"{'Tiempo Sistema (W)':<20} | {sim_W:.4f}     | {teo_W:.4f}")

Comparación con ejercicio analítico:
Lambda: 0.3333
Mu (cada servidor): 0.5000
Rho (intensidad): 0.3333
MÉTRICA              | SIMULACIÓN | TEÓRICO   
Tiempo en Fila (Wq)  | 0.2770     | 0.2500
Tiempo Sistema (W)   | 2.3230     | 2.2500
